__数据导入__

In [16]:
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class Generate_Dataset(Dataset):
    def __init__(self, path):
        self.path = path
        self.data_path = glob.glob(os.path.join(path,'input/*.npy'))  #读取data文件夹下所有.npy格式文件

    def __getitem__(self, index):
        data_path = self.data_path[index]
        data = np.load(data_path)      #读取输入数据
        tensor_data = torch.from_numpy(data)
        
        label_path = data_path.replace('input', 'label')
        label = np.load(label_path)    #读取标签数据
        tensor_label = torch.from_numpy(label)

        return tensor_data, tensor_label

    def __len__(self):
        return len(self.data_path)
# if __name__ == '__main__':
#     dataset = Generate_Dataset('../../../../Generate_dataset/Matlab_files/dataset/dataset/')
#     train_size = int(len(dataset) * 0.9)
#     validate_size = int(len(dataset) - train_size)
#     train_dataset, validate_dataset = torch.utils.data.random_split(dataset, [train_size, validate_size])

#     #print("读入数据个数为：", len(top_dataset))
#     train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
#     validate_loader = DataLoader(validate_dataset, batch_size=1, shuffle=True)
#     t = 0
#     for train, label in train_loader:
#         t += 1
#         print(train.shape)
#         print(label.shape)
#     print('共有',t,'个训练集')
    
#     n = 0
#     for validate, label in validate_loader:
#         n += 1
#         print(validate.shape)
#         print(label.shape)
#     print('共有',n,'个训练集')

In [17]:
data_size = 45
train_size = int(data_size * 0.9)
print(train_size)

40


__unet3d_parts__

In [18]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

class DoubleConv3d_init(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv3d_init, self).__init__()
        self.double_conv3d_init = nn.Sequential(
            nn.Conv3d(in_channels, out_channels=32, kernel_size=(3, 3, 3), stride=1, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(inplace=True),
            nn.Conv3d(32, out_channels, kernel_size=(3, 3, 3), stride=1, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, input):
        return self.double_conv3d_init(input)


class DoubleConv3d(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv3d, self).__init__()
        self.double_conv3d = nn.Sequential(
            nn.Conv3d(in_channels, in_channels, kernel_size=(3, 3, 3), stride=1, padding=1),
            nn.BatchNorm3d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(in_channels, out_channels, kernel_size=(3, 3, 3), stride=1, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, input):
        return self.double_conv3d(input)


class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Down,self).__init__()
        self.maxpool_conv3d = nn.Sequential(
            nn.MaxPool3d(kernel_size=2, stride=2, padding=0),
            DoubleConv3d(in_channels, out_channels)
        )

    def forward(self, input):
        return self.maxpool_conv3d(input)


class Up(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Up, self).__init__()
        self.up3d = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=True)
        self.conv = DoubleConv3d(in_channels, out_channels)

    def forward(self, input, x):  #x是接收的从encoder传过来的融合数据
        #print('input',input.shape)
        #print('x',x.shape)
        x1 = self.up3d(input)
        #print('x1',x1.shape)
        diffY = torch.tensor(x1.size()[3] - x.size()[3])
        diffX = torch.tensor(x1.size()[4] - x.size()[4])#特征融合部分
        diffZ = torch.tensor(x1.size()[2] - x.size()[2])
        #if x1.size()[3] > x.size()[3]:
        x3 = F.pad(x, (diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2,
                        diffZ // 2, diffZ - diffZ // 2))
        #print('x3',x3.shape)
        output = torch.cat([x1, x3], dim = 1)
        return self.conv(output)

class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv,self).__init__()
        self.conv1 = nn.Sequential(
                    nn.Conv3d(in_channels, out_channels, kernel_size=(1, 1, 1)))
    def forward(self, input):
        return self.conv1(input)

__unet3d_model__

In [19]:

class UNet3D(nn.Module):
    def __init__(self,in_channels, n_classes):
        super(UNet3D, self).__init__()
        self.in_channels = in_channels
        self.n_classes = n_classes

        #Encoder
        self.inc = DoubleConv3d_init(in_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)

        #Decoder
        self.up1 = Up(768, 256)
        self.up2 = Up(384, 128)
        self.up3 = Up(192, 64)
        self.outc = OutConv(64, n_classes)

    def forward(self, input):
        out1 = self.inc(input)
        print('out1.shape:',out1.shape)
        out2 = self.down1(out1)
        print('out2.shape:',out2.shape)
        out3 = self.down2(out2)
        print('out3.shape:',out3.shape)
        out4 = self.down3(out3)
        print('out4.shape:',out4.shape)
        out5 = self.up1(out4, out3)
        print('out5.shape:',out5.shape)
        out6 = self.up2(out5, out2)
        print('out6.shape:',out6.shape)
        out7 = self.up3(out6, out1)
        print('out7.shape:',out7.shape)
        logits = self.outc(out7)
        print('logits.shape:',logits.shape)
        return logits

# if __name__ == '__main__':
    
#     net = UNet3D(in_channels =3 ,n_classes=3)
#     print(net)
#     para = list(net.parameters())
#     print('parameters:', para)

__train__

In [20]:
from torch import optim
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('./loss')

def train_net(net, device, data_path, epochs=100, batch_size=1, lr=0.00001):
    dataset = Generate_Dataset(data_path)
    #划分训练集和验证集
    train_size = int(len(dataset) * 0.9)
    validate_size = int(len(dataset) - train_size)
    train_dataset, validate_dataset = torch.utils.data.random_split(dataset, [train_size, validate_size])
    #加载训练集和验证集
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    validate_loader = DataLoader(validate_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = optim.Adam(net.parameters(), lr=lr)
    criterion = nn.MSELoss()
    best_loss = float('inf')

    for epoch in range(epochs):
        print('epoch:',epoch)
        net.train()
        
        #训练数据集
        for data, label in train_loader:
            
            optimizer.zero_grad()
            data = data.to(device=device, dtype=torch.float32)
            label = label.to(device=device, dtype=torch.float32)
            pred = net(data)
            loss = criterion(pred, label)
            writer.add_scalar('trainloss',float(loss),epoch)
            writer.close()

            loss.backward()
            optimizer.step()
            print('Loss/train', loss.item())
            if loss < best_loss:
                best_loss = loss
            torch.save(net.state_dict(), 'best_model.pth')
        #验证集测试
        for validatedata, validatelabel in validate_loader:
            
            validatedata = validatedata.to(device, dtype=torch.float32)
            validatelabel = validatelabel.to(device, dtype=torch.float32)
            validatepred = net(validatedata)
            validateloss = criterion(validatepred, validatelabel)
            writer.add_scalar('validateloss',float(validateloss),epoch)
            print('Loss/validate', validateloss.item())
        
        
        
if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net = UNet3D(1, 1)
    net.to(device=device)

    data_path = "../../../../Generate_dataset/Matlab_files/dataset/dataset/"
    train_net(net, device, data_path)



epoch: 0
out1.shape: torch.Size([1, 64, 120, 120, 120])
out2.shape: torch.Size([1, 128, 60, 60, 60])
out3.shape: torch.Size([1, 256, 30, 30, 30])
out4.shape: torch.Size([1, 512, 15, 15, 15])
out5.shape: torch.Size([1, 256, 30, 30, 30])
out6.shape: torch.Size([1, 128, 60, 60, 60])
out7.shape: torch.Size([1, 64, 120, 120, 120])
logits.shape: torch.Size([1, 1, 120, 120, 120])


KeyboardInterrupt: 

__test__

In [ ]:
import glob
import numpy as np
import torch
import os



if __name__ == "__main__":
    # 选择设备，有cuda用cuda，没有就用cpu
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # 加载网络，图片单通道，分类为1。
    net = UNet3D(1, 1)
    # 将网络拷贝到deivce中
    net.to(device=device)
    # 加载模型参数
    net.load_state_dict(torch.load('best_model.pth', map_location=device))
    # 测试模式
    net.eval()
    t = 50
    # 保存结果地址
    save_res_path = os.path.join('./dataset/test/'+('%d_res.png'%(t)))
    # 转为batch为1，通道为1，大小为512*512的数组
    data = np.load('./dataset/test/topomodel_50.npy')
    data = data.reshape(1, 1, data.shape[0], data.shape[1], data.shape[2])
    # 转为tensor
    data_tensor = torch.from_numpy(data)
    # 将tensor拷贝到device中，只用cpu就是拷贝到cpu中，用cuda就是拷贝到cuda中。
    data_tensor = data_tensor.to(device=device, dtype=torch.float32)
    # 预测
    pred = net(data_tensor)
    # 提取结果
    pred = np.array(pred.data.cpu()[0])[0]
    print(pred)
    # # 处理结果
    # for i in range(pred.shape[0]):
    #     for j in range(pred.shape[1]):
    #         for k in range(pred.shape[2]):
    #             if pred[i][j][k] > 0.5:
    #                 pred[i][j][k] = 1
    #             else:
    #                 pred[i][j][k] = 0
    # 保存图片
    np.save('./dataset/test/',pred)